In [6]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

In [7]:
import pandas as pd
df = pd.read_csv("../data/clean_train_v1.csv")
df.head()

,label,tweet
0,0,"- Awww, that's a bummer. You shoulda got David..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am ..."


In [58]:
df["label"].value_counts()

label
0    798351
4    798152
Name: count, dtype: int64

In [8]:
model = SentenceTransformer("thenlper/gte-base")

/Users/junaid.shaikh/Desktop/Personal/sentiment-detection/sentx/.env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [51]:
sample = df.sample(frac=0.1, random_state=42)


In [53]:
sentences = sample["tweet"].tolist()
embeddings = model.encode(sentences)

In [54]:
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(embeddings)


KMeans(n_clusters=3, random_state=0)

In [135]:
sample[kmeans.labels_ == 0]["label"].value_counts()

label
0    41308
4    11931
Name: count, dtype: int64

In [136]:
sample[kmeans.labels_ == 1]["label"].value_counts()

label
4    51531
0    13704
Name: count, dtype: int64

In [137]:
sample[kmeans.labels_ == 2]["label"].value_counts()

label
0    24474
4    16702
Name: count, dtype: int64

In [59]:
# based on majority of votes
# cluter label - 0 -> negative
# cluter label - 1 -> positive
# cluter label - 2 -> neutral

In [126]:
df_final = df.sample(frac=0.2, random_state=42)
df_final.reset_index(inplace=True)

In [128]:
df_final.shape

(319301, 4)

In [129]:
df_final["updated_label"] = -1

In [130]:
batch_size = 100
num_batches = df_final.shape[0]//batch_size

for idx in range(num_batches):
    batch = df_final.iloc[idx*batch_size:batch_size*(idx+1)]
    sentences = batch["tweet"].to_list()
    emb = model.encode(sentences)
    preds =  kmeans.predict(emb)
    df_final.loc[idx*batch_size:batch_size*(idx+1)-1, "updated_label"] = preds
    print(f"done batch {idx}/{num_batches}")

done batch 0/3193
done batch 1/3193
done batch 2/3193
done batch 3/3193
done batch 4/3193
done batch 5/3193
done batch 6/3193
done batch 7/3193
done batch 8/3193
done batch 9/3193
done batch 10/3193
done batch 11/3193
done batch 12/3193
done batch 13/3193
done batch 14/3193
done batch 15/3193
done batch 16/3193
done batch 17/3193
done batch 18/3193
done batch 19/3193
done batch 20/3193
done batch 21/3193
done batch 22/3193
done batch 23/3193
done batch 24/3193
done batch 25/3193
done batch 26/3193
done batch 27/3193
done batch 28/3193
done batch 29/3193
done batch 30/3193
done batch 31/3193
done batch 32/3193
done batch 33/3193
done batch 34/3193
done batch 35/3193
done batch 36/3193
done batch 37/3193
done batch 38/3193
done batch 39/3193
done batch 40/3193
done batch 41/3193
done batch 42/3193
done batch 43/3193
done batch 44/3193
done batch 45/3193
done batch 46/3193
done batch 47/3193
done batch 48/3193
done batch 49/3193
done batch 50/3193
done batch 51/3193
done batch 52/3193
don

In [196]:
df_true_positive = df_final[np.logical_and(df_final["label"] == 4, df_final["updated_label"]==1)]
df_true_negative = df_final[np.logical_and(df_final["label"] == 0, df_final["updated_label"]==0)]
df_neutral = df_final[df_final["updated_label"] == 2]

In [197]:
df_true_positive["num_words"] = df_true_positive["tweet"].apply(lambda x : len(x.split()))
df_true_negative["num_words"] = df_true_negative["tweet"].apply(lambda x : len(x.split()))
df_neutral["num_words"] = df_neutral["tweet"].apply(lambda x : len(x.split()))

/var/folders/6w/s9mj_vqn2dx406sgb8jryggc0000gp/T/ipykernel_31323/603583717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_positive["num_words"] = df_true_positive["tweet"].apply(lambda x : len(x.split()))
/var/folders/6w/s9mj_vqn2dx406sgb8jryggc0000gp/T/ipykernel_31323/603583717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_negative["num_words"] = df_true_negative["tweet"].apply(lambda x : len(x.split()))
/var/folders/6w/s9mj_vqn2dx406sgb8jryggc0000gp/T/ipykernel_31323/603583717.p

In [198]:
df_true_positive=df_true_positive[df_true_positive["num_words"] > 1]
df_true_negative=df_true_negative[df_true_negative["num_words"] > 1]
df_neutral=df_neutral[df_neutral["num_words"] > 1]

In [199]:
df_neutral.sample(n=10)["tweet"].tolist()

['LOL! I have to have brunch first. Well done on your bike ride. Am impressed',
 'im in a hotel now. and i heard lotsa scary stories and im super effin scared i cant even look in the mirror. haha!',
 'We are SO ready!! Even got a new clicker for my PowerPoint and a new projector - how swisho are we! Not many sleeps to go now',
 'played wii whole afternoon. whataway to spend a rainy bum weekend',
 '13 days until I see my hubby!! I miss him',
 "haha! It's not really morning, well i guess it is, but ya, i didn't acually wake up this late, don't worry.",
 'i wish i had milk right now.. i am craving some cereal.',
 'going to bed! early day tomorrow!',
 'i need to sleep cos i have to get up at 6am but im too excited and keeps posting tweets. go away harry let me go to bed haha',
 "i'm going to bed. school tomorrow"]

In [200]:
df_true_negative.sample(n=10)["tweet"].tolist()

["i need to escape big time...i feel like i'm being held back",
 'Had a dream last night my friends &amp; I went to high school musical prom but nobody asked me2 dance cus all the cute boys thought I was fug',
 'Drag Me To Hell was fairly interesting. Scared the crap out of me',
 "I'm back seems GF didn't wait for me I took too long",
 'Atrocious start for Rubens tho',
 "I dunno how to do that and I don't want to go to the party saturday because I just don't want to get hurt again",
 "hmmm, have I chosen wisely....don't like this collar much",
 'reading the draft of Midnight Sun... not as good as i expected sadly',
 "I know - not surprised though as she does f all! They probably wouldn't do anything would they The poor baby",
 'what sucks about grades is I had a teacher once who gave me an A- just to wreck the 4.0 he even said so... Oh well right!']

In [201]:
df_true_positive.sample(n=10)["tweet"].tolist()

["Britain's Got Talent is amazing tonight. Diversity to win!",
 'cool! good luck ;)',
 "Watching MTV Movie Awards. Instead of updating twitter, i'm updating my sister. lol. Robert Pattinson was so cute tonight.",
 "Nothing like mom's cooking unless u know all the recipes &amp; make 'em better! Hello to the Fab 5+2! Spaghetti 4 me 2nite!",
 "that video was funny i'm just signing up now and i'll vote for you!! x",
 'True multitasking Or is it rapid refocusing ;-)',
 'Has the most amazing and attractive boyfriend ever',
 'Great idea. I have just bought some credits!',
 "submitting a photo to NBC 4 that I took of a soldier @ New Ro's Memorial Day parade ... I hope they use it",
 'I know its a Block Party but would love to hear a DH original if you get a chance']

In [202]:
sample_size = 80000

df_merged = pd.concat([df_true_negative.sample(n=sample_size), df_true_positive.sample(n=sample_size), df_neutral.sample(n=sample_size)])

In [203]:
df_merged.head()

,index,label,tweet,updated_label,num_words
301210,501532,0,.what the hell is happening...,0,5
68411,609271,0,"I know, I would have , but space is tight here...",0,12
142404,395691,0,i think i may have lost a friend,0,8
220757,341582,0,if fed up with the NSW v QLD game,0,9
204432,12308,0,Sudbury's never gonna win,0,4


In [204]:
df_merged.to_csv("../data/train_data-v1.csv")

In [ ]:
predicted_label = []
actual_labels = []



